In [1]:
# Import the dependencies
import pandas as pd
import gmaps
import requests

# Import the API key
from config import g_key
from config import key

In [2]:
# Store the CSV into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Upernavik,GL,2021-01-19 02:13:58,72.7868,-56.1549,7.07,78,10,7.83
1,1,Niort,FR,2021-01-19 02:09:18,46.3333,-0.3333,34.00,88,96,8.50
2,2,Luderitz,NaN,2021-01-19 02:13:58,-26.6481,15.1594,64.92,84,49,11.97
3,3,Rikitea,PF,2021-01-19 02:13:58,-23.1203,-134.9692,78.48,74,72,14.41
4,4,Saint-Philippe,RE,2021-01-19 02:13:58,-21.3585,55.7679,78.80,69,0,4.61


In [3]:
# Get data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use Google API key
gmaps.configure(api_key=g_key)
gmaps.configure(api_key=key)

In [5]:
# Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
# Or add [max(temp, 0) for temp in max_temp] right in the code below

In [17]:
# Heatmap of temperature
# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
# max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, max_intensity=300, point_radius=4, dissipating=False)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a min and max temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 71


In [10]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
46,46,Altamira,BR,2021-01-19 02:14:03,-3.2033,-52.2064,70.23,95,100,1.25
90,90,Huarmey,PE,2021-01-19 02:14:07,-10.0681,-78.1522,70.47,83,100,7.11
235,235,Viedma,AR,2021-01-19 02:10:32,-40.8135,-62.9967,70.74,38,0,8.12
440,440,Alto Araguaia,BR,2021-01-19 02:14:47,-17.3147,-53.2153,70.29,94,100,2.82
474,474,Mundo Nuevo,MX,2021-01-19 02:14:50,18.7000,-95.5000,70.97,82,97,9.46
506,506,Tacuati,PY,2021-01-19 02:14:53,-23.4500,-56.5833,70.27,76,44,3.11


In [11]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
46,Altamira,BR,70.23,-3.2033,-52.2064,
90,Huarmey,PE,70.47,-10.0681,-78.1522,
235,Viedma,AR,70.74,-40.8135,-62.9967,
440,Alto Araguaia,BR,70.29,-17.3147,-53.2153,
474,Mundo Nuevo,MX,70.97,18.7000,-95.5000,
506,Tacuati,PY,70.27,-23.4500,-56.5833,


In [13]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot data
fig

Figure(layout=FigureLayout(height='420px'))